In [1]:
!pip install  numpy


In [84]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"


In [8]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17129, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17129 (delta 51), reused 27 (delta 22), pack-reused 17043 (from 2)
Receiving objects: 100% (17129/17129), 15.80 MiB | 27.56 MiB/s, done.
Resolving deltas: 100% (11750/11750), done.


In [9]:
%cd yolov5
!pip install -r /kaggle/working/yolov5/requirements.txt

/kaggle/working/yolov5/yolov5


In [3]:
import os
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import pytorch_lightning as pl

# Define paths
video_dir = "/kaggle/input/cholec-train-data/CholecT50/videos"
label_dir = "/kaggle/input/cholec-train-data/CholecT50/labels"

# Training and testing video sub-folder names
train_videos = ["VID01", "VID02", "VID04", "VID05", "VID06", "VID08", "VID10", "VID12", "VID13", "VID14"]
test_videos = ["VID92", "VID96", "VID103", "VID110", "VID111"]

def parse_annotations(label_path):
    """Parse JSON annotations and extract bounding boxes and triplet data."""
    with open(label_path, 'r') as file:
        data = json.load(file)
    annotations = data['annotations']

    parsed_data = []
    for frame_id, triplets in annotations.items():
        for triplet in triplets:
            parsed_data.append({
                'frame_id': int(frame_id),
                'triplet_id': triplet[0],
                'instrument_id': triplet[1],
                'verb_id': triplet[7],
                'target_id': triplet[8],
                'phase_id': triplet[14],
                'bbox': triplet[3:7]
            })
    return parsed_data

def create_dataset(video_list, video_dir, label_dir):
    """Create a DataFrame for the dataset."""
    data = []
    for video_id in tqdm(video_list, desc="Processing Videos"):
        video_path = Path(video_dir) / video_id
        label_path = Path(label_dir) / f"{video_id}.json"

        annotations = parse_annotations(label_path)
        for entry in annotations:
            frame_path = video_path / f"{entry['frame_id']}.png"
            data.append({
                'video_id': video_id,
                'frame_path': str(frame_path),
                'triplet_id': entry['triplet_id'],
                'instrument_id': entry['instrument_id'],
                'verb_id': entry['verb_id'],
                'target_id': entry['target_id'],
                'phase_id': entry['phase_id'],
                'bbox': entry['bbox']
            })
    return pd.DataFrame(data)

# Prepare Train and Test DataFrames
train_df = create_dataset(train_videos, video_dir, label_dir)
test_df = create_dataset(test_videos, video_dir, label_dir)

# Save DataFrames to CSV for verification
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

# Print DataFrame samples
print("Training Data Sample:")
train_df.head()
print("\nTesting Data Sample:")
test_df.head()


Processing Videos: 100%|██████████| 5/5 [00:00<00:00, 33.17it/s]


Training Data Sample:

Testing Data Sample:


,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,"[-1, -1, -1, -1]"
1,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,"[-1, -1, -1, -1]"
2,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,"[-1, -1, -1, -1]"
3,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,"[-1, -1, -1, -1]"
4,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,"[-1, -1, -1, -1]"


In [4]:
train_df.head()

,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[-1, -1, -1, -1]"
1,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[-1, -1, -1, -1]"
2,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[-1, -1, -1, -1]"
3,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[-1, -1, -1, -1]"
4,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[-1, -1, -1, -1]"


In [4]:
train_df.iloc()[0][1]

<ipython-input-4-418e02f071fb>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_df.iloc()[0][1]


'/kaggle/input/cholec-train-data/CholecT50/videos/VID01/0.png'

In [5]:
import xml.etree.ElementTree as ET
import os

annotations_dir = "/kaggle/input/finetuning/m2cai16-tool-locations/Annotations"
unique_classes = set()

for annotation_file in os.listdir(annotations_dir)[:10]:  # Check first 10 files
    annotation_path = os.path.join(annotations_dir, annotation_file)
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    for obj in root.findall("object"):
        class_name = obj.find("name").text
        unique_classes.add(class_name)

print("Classes found in XML annotations:")
print(unique_classes)


Classes found in XML annotations:
{'Bipolar', 'Irrigator', 'SpecimenBag', 'Clipper', 'Grasper', 'Scissors'}


In [6]:
# import os
# import cv2
# import xml.etree.ElementTree as ET
# from tqdm import tqdm
# from sklearn.model_selection import train_test_split

# # Define paths
# annotations_dir = "/kaggle/input/finetuning/m2cai16-tool-locations/Annotations"
# images_dir = "/kaggle/input/finetuning/m2cai16-tool-locations/JPEGImages"
# class_list_path = "/kaggle/input/finetuning/m2cai16-tool-locations/class_list.txt"
# output_dir = "/kaggle/working/finetuning_yolo_dataset"

# # Load and clean class names (strip numeric prefixes)
# with open(class_list_path, "r") as file:
#     classes = [line.strip().split(' ', 1)[1] for line in file.readlines()]

# print("Processed class names from class_list.txt:")
# print(classes)

# # Validate class names against XML
# xml_classes = set()
# for xml_file in os.listdir(annotations_dir):
#     tree = ET.parse(os.path.join(annotations_dir, xml_file))
#     for obj in tree.getroot().findall("object"):
#         xml_classes.add(obj.find("name").text)

# print("Classes found in XML annotations:")
# print(xml_classes)

# # Check if all XML classes exist in class_list.txt
# missing_classes = xml_classes - set(classes)
# if missing_classes:
#     print(f"Warning: Missing classes in class_list.txt: {missing_classes}")
# else:
#     print("All XML classes are present in class_list.txt.")

# # Ensure output directories exist
# os.makedirs(output_dir, exist_ok=True)
# os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
# os.makedirs(f"{output_dir}/labels/val", exist_ok=True)
# os.makedirs(f"{output_dir}/images/train", exist_ok=True)
# os.makedirs(f"{output_dir}/images/val", exist_ok=True)

# # Function to convert Pascal VOC annotations to YOLO format
# def voc_to_yolo(annotation_path, image_width, image_height):
#     tree = ET.parse(annotation_path)
#     root = tree.getroot()
#     yolo_annotations = []

#     for obj in root.findall("object"):
#         class_name = obj.find("name").text
#         if class_name not in classes:
#             continue  # Skip unknown classes

#         class_id = classes.index(class_name)
#         bbox = obj.find("bndbox")
#         xmin = float(bbox.find("xmin").text)
#         ymin = float(bbox.find("ymin").text)
#         xmax = float(bbox.find("xmax").text)
#         ymax = float(bbox.find("ymax").text)

#         # Convert to YOLO format
#         x_center = ((xmin + xmax) / 2) / image_width
#         y_center = ((ymin + ymax) / 2) / image_height
#         width = (xmax - xmin) / image_width
#         height = (ymax - ymin) / image_height
#         yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

#     return yolo_annotations

# # Split data into train and val sets
# annotation_files = os.listdir(annotations_dir)
# train_files, val_files = train_test_split(annotation_files, test_size=0.2, random_state=42)

# # Process and save annotations in YOLO format
# for split, files in [("train", train_files), ("val", val_files)]:
#     for annotation_file in tqdm(files, desc=f"Processing {split} data"):
#         annotation_path = os.path.join(annotations_dir, annotation_file)
#         image_file = annotation_file.replace(".xml", ".jpg")
#         image_path = os.path.join(images_dir, image_file)
        
#         # Get image dimensions
#         if not os.path.exists(image_path):
#             print(f"Image not found: {image_path}")
#             continue
        
#         image = cv2.imread(image_path)
#         height, width, _ = image.shape

#         # Convert annotations
#         yolo_annotations = voc_to_yolo(annotation_path, width, height)
        
#         # Save annotations
#         with open(f"{output_dir}/labels/{split}/{image_file.replace('.jpg', '.txt')}", "w") as file:
#             file.write("\n".join(yolo_annotations))
        
#         # Copy images to the corresponding directory
#         os.system(f"cp {image_path} {output_dir}/images/{split}/")

# # Save dataset.yaml for YOLOv5 training
# dataset_yaml = f"""
# train: {output_dir}/images/train
# val: {output_dir}/images/val

# nc: {len(classes)}
# names: {classes}
# """
# with open(f"{output_dir}/dataset.yaml", "w") as file:
#     file.write(dataset_yaml)


In [7]:
import os

# Disable W&B visualization by setting the environment variable
os.environ["WANDB_MODE"] = "disabled"


In [8]:
# !rm /kaggle/working/finetuning_yolo_dataset/labels/train.cache


In [9]:
# !python train.py --img 640 --batch 16 --epochs 25 --data /kaggle/working/finetuning_yolo_dataset/dataset.yaml --weights /kaggle/input/model-weight-1/best.pt --project /kaggle/working/runs --name finetune_yolov5


In [10]:
import os
from tqdm import tqdm
import torch
from yolov5.utils.general import non_max_suppression
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.dataloaders import LoadImages
from yolov5.utils.torch_utils import select_device

# Ensure all paths have the correct 6-digit format
train_df['frame_path'] = train_df['frame_path'].apply(
    lambda x: os.path.join(
        os.path.dirname(x),
        f"{int(os.path.basename(x).split('.')[0]):06d}.png"
    )
)

# Validate paths
print("Validating file paths...")
missing_files = train_df[~train_df['frame_path'].apply(os.path.exists)]
if not missing_files.empty:
    print(f"Warning: {len(missing_files)} files are missing.")
    print(missing_files.head())
else:
    print("All file paths are valid.")

# Set model path and device
model_path = "/kaggle/input/fintune-weight/finetuned.pt"
device = select_device('cuda' if torch.cuda.is_available() else 'cpu')

# Load YOLOv5 model
model = DetectMultiBackend(model_path, device=device)

# Function to get bounding boxes for instruments
def get_bounding_boxes(image_path):
    dataset = LoadImages(image_path)
    results = []
    
    for path, img, img0, vid_cap, _ in dataset:
        img = torch.from_numpy(img).to(device).float() / 255.0  # Normalize
        if len(img.shape) == 3:
            img = img.unsqueeze(0)  # Add batch dimension
        
        # Perform inference
        pred = model(img)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
        
        for det in pred:
            if det is not None and len(det):
                for *box, conf, cls in det:
                    results.append(box)  # Add bounding box coordinates
    return results

# Apply YOLOv5 on train_df to get bounding boxes
bbox_list = []
for image_path in tqdm(train_df['frame_path'], desc="Processing Images"):
    bboxes = get_bounding_boxes(image_path)
    bbox_list.append(bboxes)

# Save bounding box coordinates in train_df
train_df['bbox'] = bbox_list

# Save train_df with bounding boxes
train_df.to_csv("/kaggle/working/train_df_with_bboxes.csv", index=False)

print("Bounding boxes added to train_df and saved as train_df_with_bboxes.csv.")


ModuleNotFoundError: No module named 'yolov5'

In [ ]:
import os
from tqdm import tqdm
import torch
from yolov5.utils.general import non_max_suppression
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.dataloaders import LoadImages
from yolov5.utils.torch_utils import select_device

# Ensure all paths in test_df have the correct 6-digit format
test_df['frame_path'] = test_df['frame_path'].apply(
    lambda x: os.path.join(
        os.path.dirname(x),
        f"{int(os.path.basename(x).split('.')[0]):06d}.png"
    )
)

# Validate paths in test_df
print("Validating file paths in test_df...")
missing_files_test = test_df[~test_df['frame_path'].apply(os.path.exists)]
if not missing_files_test.empty:
    print(f"Warning: {len(missing_files_test)} files are missing in test_df.")
    print(missing_files_test.head())
else:
    print("All file paths in test_df are valid.")

# Set model path and device
model_path = "/kaggle/input/fintune-weight/finetuned.pt"
device = select_device('cuda' if torch.cuda.is_available() else 'cpu')

# Load YOLOv5 model
model = DetectMultiBackend(model_path, device=device)

# Function to get bounding boxes for instruments
def get_bounding_boxes(image_path):
    dataset = LoadImages(image_path)
    results = []
    
    for path, img, img0, vid_cap, _ in dataset:
        img = torch.from_numpy(img).to(device).float() / 255.0  # Normalize
        if len(img.shape) == 3:
            img = img.unsqueeze(0)  # Add batch dimension
        
        # Perform inference
        pred = model(img)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
        
        for det in pred:
            if det is not None and len(det):
                for *box, conf, cls in det:
                    results.append(box)  # Add bounding box coordinates
    return results

# Apply YOLOv5 on test_df to get bounding boxes
bbox_list_test = []
for image_path in tqdm(test_df['frame_path'], desc="Processing Test Images"):
    bboxes = get_bounding_boxes(image_path)
    bbox_list_test.append(bboxes)

# Save bounding box coordinates in test_df
test_df['bbox'] = bbox_list_test

# Save test_df with bounding boxes
test_df.to_csv("/kaggle/working/test_df_with_bboxes.csv", index=False)

print("Bounding boxes added to test_df and saved as test_df_with_bboxes.csv.")


In [5]:
import pandas as pd

# Specify the path to the CSV file
csv_file_path = "/kaggle/input/procesed-train-data/train_df_with_bboxes.csv"

# Load the CSV file into a DataFrame
train_df_m = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded
train_df_m.head()


,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,1.0,0,"[[tensor(285.87274), tensor(16.54350), tensor(..."
1,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,1.0,0,"[[tensor(179.74133), tensor(19.72769), tensor(..."
2,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,1.0,0,"[[tensor(140.20927), tensor(19.87531), tensor(..."
3,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,1.0,0,[]
4,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,1.0,0,[]


In [6]:
# Copy the 'bbox' column from train_df_m to train_df
train_df['bbox'] = train_df_m['bbox']

# Save train_df to a new file (e.g., as a CSV)
# train_df.to_csv('train_df_updated.csv', index=False)


In [7]:
train_df.head()

,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(285.87274), tensor(16.54350), tensor(..."
1,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(179.74133), tensor(19.72769), tensor(..."
2,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(140.20927), tensor(19.87531), tensor(..."
3,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,[]
4,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,[]


In [8]:
print(train_df.columns)


Index(['video_id', 'frame_path', 'triplet_id', 'instrument_id', 'verb_id',
       'target_id', 'phase_id', 'bbox'],
      dtype='object')


In [9]:
import pandas as pd

# Specify the path to the CSV file
csv_file_path = "/kaggle/input/procesed-test-data/test_df_with_bboxes.csv"

# Load the CSV file into a DataFrame
test_df_m = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded
test_df_m.head()


,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,1.0,0,[]
1,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,1.0,0,[]
2,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,1.0,0,[]
3,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,1.0,0,[]
4,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,1.0,0,[]


In [10]:
# Copy the 'bbox' column from train_df_m to train_df
test_df['bbox'] = test_df_m['bbox']

# Save train_df to a new file (e.g., as a CSV)
# test_df.to_csv('test_df_updated.csv', index=False)


In [11]:
   test_df.head()

,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,[]
1,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,[]
2,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,[]
3,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,[]
4,VID92,/kaggle/input/cholec-train-data/CholecT50/vide...,-1,-1,-1,-1,0,[]


In [18]:
train_df.iloc[0][1]

<ipython-input-18-3841664e9223>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_df.iloc[0][1]


'/kaggle/input/cholec-train-data/CholecT50/videos/VID01/0.png'

In [12]:
train_df['target_id'].nunique()

16

In [52]:
test_df.columns

Index(['video_id', 'frame_path', 'triplet_id', 'instrument_id', 'verb_id',
       'target_id', 'phase_id', 'bbox'],
      dtype='object')

In [36]:
# import torch
# import torch.nn as nn
# from torchvision.models import resnet50
# import pytorch_lightning as pl
# import os

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # For debugging CUDA errors
# os.environ["OMP_NUM_THREADS"] = "1"

# # Load classes from file
# def load_classes_from_file(file_path):
#     with open(file_path, 'r') as file:
#         classes = file.read().splitlines()  # Read all lines and remove newlines
#     return classes

# # Paths to the files containing the class mappings
# instrument_classes_path = '/kaggle/input/dict-mapping/dict/instrument.txt'
# target_classes_path = '/kaggle/input/dict-mapping/dict/target.txt'
# verb_classes_path = '/kaggle/input/dict-mapping/dict/verb.txt'
# triplet_classes_path = '/kaggle/input/dict-mapping/dict/triplet.txt'

# # Load classes from the files
# instrument_classes = load_classes_from_file(instrument_classes_path)
# target_classes = load_classes_from_file(target_classes_path)
# verb_classes = load_classes_from_file(verb_classes_path)
# triplet_classes = load_classes_from_file(triplet_classes_path)

# # Define MultiTaskTripletModel
# class MultiTaskTripletModel(nn.Module):
#     def __init__(self, num_instruments, num_verbs, num_targets, num_triplets):
#         super(MultiTaskTripletModel, self).__init__()

#         # Load a pre-trained ResNet-50 backbone
#         self.backbone = resnet50(pretrained=True)
#         self.backbone.fc = nn.Identity()  # Remove the fully connected layer

#         # Fully connected layers for each task
#         self.instrument_head = nn.Sequential(
#             nn.Linear(2048, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, num_instruments)
#         )

#         self.verb_head = nn.Sequential(
#             nn.Linear(2048, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, num_verbs)
#         )

#         self.target_head = nn.Sequential(
#             nn.Linear(2048, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, num_targets)
#         )

#         self.triplet_head = nn.Sequential(
#             nn.Linear(2048, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, num_triplets)
#         )

#     def forward(self, x):
#         features = self.backbone(x)  # Extract features from the backbone

#         instrument_logits = self.instrument_head(features)
#         verb_logits = self.verb_head(features)
#         target_logits = self.target_head(features)
#         triplet_logits = self.triplet_head(features)

#         return instrument_logits, verb_logits, target_logits, triplet_logits

# # Define MultiTaskTripletLightningModel
# class MultiTaskTripletLightningModel(pl.LightningModule):
#     def __init__(self, num_instruments, num_verbs, num_targets, num_triplets):
#         super(MultiTaskTripletLightningModel, self).__init__()
#         self.model = MultiTaskTripletModel(num_instruments, num_verbs, num_targets, num_triplets)
#         self.criterion = nn.CrossEntropyLoss()

#     def forward(self, x):
#         return self.model(x)

#     def training_step(self, batch, batch_idx):
#         images, labels_dict, _ = batch  # Unpack batch
#         instrument_logits, verb_logits, target_logits, triplet_logits = self(images)

#         # Compute losses
#         instrument_loss = self.criterion(instrument_logits, labels_dict["instrument_id"])
#         verb_loss = self.criterion(verb_logits, labels_dict["verb_id"])
#         target_loss = self.criterion(target_logits, labels_dict["target_id"])
#         triplet_loss = self.criterion(triplet_logits, labels_dict["triplet_id"])

#         # Total loss
#         total_loss = instrument_loss + verb_loss + target_loss + triplet_loss
#         self.log("train_loss", total_loss)

#         return total_loss

#     def validation_step(self, batch, batch_idx):
#         images, labels_dict, _ = batch
#         instrument_logits, verb_logits, target_logits, triplet_logits = self(images)
        
#         # Ensure logits are float32
#         instrument_logits = instrument_logits.float()
#         verb_logits = verb_logits.float()
#         target_logits = target_logits.float()
#         triplet_logits = triplet_logits.float()

#         # Ensure the labels are long (int64) type
#         instrument_labels = labels_dict["instrument_id"].long()
#         verb_labels = labels_dict["verb_id"].long()
#         target_labels = labels_dict["target_id"].long()
#         triplet_labels = labels_dict["triplet_id"].long()

#         # Compute losses
#         loss_instrument = self.criterion(instrument_logits, instrument_labels)
#         loss_verb = self.criterion(verb_logits, verb_labels)
#         loss_target = self.criterion(target_logits, target_labels)
#         loss_triplet = self.criterion(triplet_logits, triplet_labels)

#         # Total loss
#         total_loss = loss_instrument + loss_verb + loss_target + loss_triplet
#         self.log("val_loss", total_loss)

#         return total_loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=1e-3)


# # # Trainer Setup
# # trainer = pl.Trainer(max_epochs=10, gpus=1)  # Add GPU if available

# # # Fit Model
# # trainer.fit(lightning_model, datamodule=data_module)


In [15]:
import torch
def preprocess_bbox_column(dataframe):
    """
    Converts the bbox column in the DataFrame from strings to lists of tensors.

    Args:
        dataframe (pd.DataFrame): DataFrame with a 'bbox' column.

    Returns:
        pd.DataFrame: DataFrame with the bbox column preprocessed.
    """
    def parse_bbox(bbox_str):
        if isinstance(bbox_str, str):
            try:
                # Parse the string representation of bbox manually
                bbox_list = eval(bbox_str.replace("tensor(", "").replace(")", ""))
                # Convert all items in the list to tensors
                return [torch.tensor(b, dtype=torch.float32) for b in bbox_list]
            except Exception as e:
                raise ValueError(f"Error parsing bbox: {bbox_str}. Error: {e}")
        elif isinstance(bbox_str, list):
            # Ensure all elements are tensors
            return [torch.tensor(b, dtype=torch.float32) for b in bbox_str]
        else:
            return []  # Handle empty bbox

    dataframe["bbox"] = dataframe["bbox"].apply(parse_bbox)
    return dataframe


In [16]:
train_df = preprocess_bbox_column(train_df)
test_df = preprocess_bbox_column(test_df)


In [17]:
train_df['verb_id'] = train_df['verb_id'].replace(-1, 9)
train_df['target_id'] = train_df['target_id'].replace(-1, 14)



In [18]:
train_df['target_id'].nunique()

15

In [19]:
import torch
torch.cuda.empty_cache()


In [20]:
train_df= train_df[:100]
test_df= test_df[:100]

In [21]:
train_df.head()

,video_id,frame_path,triplet_id,instrument_id,verb_id,target_id,phase_id,bbox
0,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(285.8727), tensor(16.5435), tensor(38..."
1,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(179.7413), tensor(19.7277), tensor(31..."
2,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,"[[tensor(140.2093), tensor(19.8753), tensor(27..."
3,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,[]
4,VID01,/kaggle/input/cholec-train-data/CholecT50/vide...,7,0,0,0,0,[]


In [29]:
train_df.iloc[0][7]

<ipython-input-29-c081e571bf05>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_df.iloc[0][7]


[tensor([285.8727,  16.5435, 381.9036,  66.8849])]

In [22]:
# Define the input and output file paths
input_file_path = '/kaggle/input/dict-mapping/dict/instrument.txt'
output_file_path = '/kaggle/working/instrument_updated.txt'

# Open the input file and read its contents
with open(input_file_path, 'r') as file:
    # Read the file content as a string
    data = file.read()

# Convert the file content into a dictionary (since the format appears dictionary-like)
instrument_dict = {
    0: 'grasper',
    1: 'bipolar',
    2: 'hook',
    3: 'scissors',
    4: 'clipper',
    5: 'irrigator'
}

# Add the new class to the dictionary
instrument_dict[6] = 'null_instrument'

# Now, save the updated dictionary to a new file in /kaggle/working
with open(output_file_path, 'w') as file:
    # Convert the dictionary into a string and write it back to the file
    for key, value in instrument_dict.items():
        file.write(f"{key}:{value}\n")

print(f"New class '6: null_instrument' added and file saved as {output_file_path}!")


New class '6: null_instrument' added and file saved as /kaggle/working/instrument_updated.txt!


In [23]:
# Define the input and output file paths
input_file_path = '/kaggle/input/dict-mapping/dict/triplet.txt'
output_file_path = '/kaggle/working/triplet_updated.txt'

# Open the input file and read its contents
with open(input_file_path, 'r') as file:
    # Read the file content as a string
    data = file.readlines()

# Add the new class to the data
new_class = "100:null_instrument,null_verb,null_target\n"
data.append(new_class)

# Now, save the updated content to the new file in /kaggle/working
with open(output_file_path, 'w') as file:
    file.writelines(data)

print(f"New class '100:null_instrument,null_verb,null_target' added and file saved as {output_file_path}!")


New class '100:null_instrument,null_verb,null_target' added and file saved as /kaggle/working/triplet_updated.txt!


In [24]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
import pandas as pd
from PIL import Image
import numpy as np
import os

# # Disable GPU execution
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# tf.config.set_visible_devices([], 'GPU')  # Disables all GPUs
# Ensure TensorFlow GPU setup is done before any operations
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    # Set visible device to first GPU
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    # Enable memory growth for the first GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Set device for torch as well
    print("GPU detected and memory growth enabled.")
else:
    print("No GPU detected, using CPU.")
    device = torch.device('cpu')

# Load class mappings
def load_classes_from_file(file_path):
    with open(file_path, 'r') as file:
        classes = file.read().splitlines()
    return classes

instrument_classes = load_classes_from_file('/kaggle/working/instrument_updated.txt')
target_classes = load_classes_from_file('/kaggle/input/dict-mapping/dict/target.txt')
verb_classes = load_classes_from_file('/kaggle/input/dict-mapping/dict/verb.txt')
triplet_classes = load_classes_from_file('/kaggle/working/triplet_updated.txt')
# Data loading and preprocessing with bbox handling

GPU detected and memory growth enabled.


In [38]:
!pip install --upgrade tensorflow


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.1 MB/s eta 0:00:0000:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have te

In [36]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'


In [32]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [34]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  2


In [49]:
test_df.iloc[0]

video_id                                                     VID92
frame_path       /kaggle/input/cholec-train-data/CholecT50/vide...
triplet_id                                                      -1
instrument_id                                                   -1
verb_id                                                         -1
target_id                                                       -1
phase_id                                                         0
bbox                                                            []
Name: 0, dtype: object

In [29]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"CUDA version: {tf.version.COMPILER_VERSION}")


TensorFlow version: 2.17.0
CUDA version: Ubuntu Clang 17.0.6 (++20231208085846+6009708b4367-1~exp1~20231208085949.74)


In [47]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Concatenate, Lambda
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from PIL import Image
from tensorflow.keras.layers import Resizing
import torch
from tensorflow.keras.layers import Layer
tf.debugging.set_log_device_placement(True)

# Preprocess image function
def preprocess_image(image_path, label, num_instruments, num_verbs, num_targets, num_triplets):
    """
    Load and preprocess the image, ensuring proper formatting, resizing, normalization, 
    and handling of missing triplet IDs.
    """
    filename = os.path.basename(image_path)
    image_id = int(filename.split('.')[0])
    padded_image_id = f"{image_id:06d}"  # Ensure zero-padded
    corrected_image_path = os.path.join(os.path.dirname(image_path), f"{padded_image_id}.png")

    # Load and preprocess the image
    try:
        # Load the image using PIL
        image = Image.open(corrected_image_path).convert("RGB")
        
        # Resize the image to 224x224 using LANCZOS
        image = image.resize((224, 224), Image.LANCZOS)
        
        # Convert to numpy array and normalize
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    except FileNotFoundError:
        print(f"Warning: Image not found: {corrected_image_path}")
        # Create a blank 224x224 RGB image as fallback
        image = np.zeros((224, 224, 3), dtype=np.float32)

    # Handle bounding boxes (ensure they are within [0, 1])
    bbox = label.get("bbox", [0.0, 0.0, 1.0, 1.0])  # Default to full image if bbox missing
    bbox = np.clip(np.array(bbox) / 224.0, 0.0, 1.0)  # Normalize and clip to [0, 1]

    # Handle missing or invalid labels
    label["instrument_id"] = 6 if label.get("instrument_id", -1) == -1 else max(label.get("instrument_id", -1), 0)
    label["verb_id"] = max(label.get("verb_id", -1), 0)
    label["target_id"] = max(label.get("target_id", -1), 0)
    label["triplet_id"] = 100 if label.get("triplet_id", -1) == -1 else max(label.get("instrument_id", -1), 0)

    return image, bbox, label

def create_tf_dataset(dataframe, batch_size, num_instruments, num_verbs, num_targets, num_triplets):
    def generator():
        for _, row in dataframe.iterrows():
            try:
                bbox = row.get("bbox", [0.0, 0.0, 1.0, 1.0])
                if len(bbox) != 4:
                    bbox = [0.0, 0.0, 1.0, 1.0]  # Default bounding box
    
                image, bbox, labels = preprocess_image(
                    row["frame_path"], 
                    {
                        "instrument_id": int(row["instrument_id"]),
                        "verb_id": int(row["verb_id"]),
                        "target_id": int(row["target_id"]),
                        "triplet_id": int(row["triplet_id"]),
                        "bbox": bbox
                    }, 
                    num_instruments, num_verbs, num_targets, num_triplets
                )
    
                bbox = np.array(bbox, dtype=np.float32)
                if bbox.shape != (4,):
                    print(f"Invalid bbox shape: {bbox.shape}, using default [0.0, 0.0, 1.0, 1.0]")
                    bbox = np.array([0.0, 0.0, 1.0, 1.0], dtype=np.float32)
    
                yield (image, bbox), (
                    labels["instrument_id"],
                    labels["verb_id"],
                    labels["target_id"],
                    labels["triplet_id"]
                )
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            (
                tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(4,), dtype=tf.float32),
            ),
            (
                tf.TensorSpec(shape=(), dtype=tf.int32),
                tf.TensorSpec(shape=(), dtype=tf.int32),
                tf.TensorSpec(shape=(), dtype=tf.int32),
                tf.TensorSpec(shape=(), dtype=tf.int32),
            )
        )
    )
    return dataset.repeat().batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Custom Layer for Scaling Bounding Box
class ScaleBBox(Layer):
    def __init__(self, target_size):
        super(ScaleBBox, self).__init__()
        self.target_size = target_size

    def call(self, bbox):
        x_min, y_min, x_max, y_max = tf.unstack(bbox, axis=-1)
        width = self.target_size[1]
        height = self.target_size[0]
        
        scaled_bbox = tf.stack([
            x_min * width,
            y_min * height,
            x_max * width,
            y_max * height
        ], axis=-1)
        
        return scaled_bbox

class CropToBBox(Layer):
    def __init__(self):
        super(CropToBBox, self).__init__()

    def call(self, inputs):
        image, bbox = inputs

        # Debug statement for checking the input sizes
        print(f"Image shape before crop: {tf.shape(image)}")
        print(f"Bounding box shape: {tf.shape(bbox)}")

        # Cast height and width to float32 for the multiplication with float values in bbox
        height = tf.cast(tf.shape(image)[1], tf.float32)
        width = tf.cast(tf.shape(image)[2], tf.float32)

        y_min, x_min, y_max, x_max = tf.split(bbox, num_or_size_splits=4, axis=-1)
        
        # Normalize bbox values
        y_min = y_min / height
        x_min = x_min / width
        y_max = y_max / height
        x_max = x_max / width

        normalized_bbox = tf.concat([y_min, x_min, y_max, x_max], axis=-1)

        # Use tf.map_fn to handle batch processing
        cropped_images = tf.map_fn(
            lambda i: self._crop_and_resize(image[i], normalized_bbox[i], height, width),
            elems=tf.range(tf.shape(image)[0]), dtype=tf.float32
        )

        # Debug statement for output shape
        print(f"Cropped image shape: {tf.shape(cropped_images)}")

        return cropped_images

    def _crop_and_resize(self, image, bbox, height, width):
        y_min, x_min, y_max, x_max = tf.unstack(bbox)
        
        # Cast height and width to float32 for calculation purposes
        height = tf.cast(height, tf.float32)
        width = tf.cast(width, tf.float32)
    
        # Use tf.cast for indexing (conversion from float to int)
        y_min = tf.cast(y_min * height, tf.int32)
        x_min = tf.cast(x_min * width, tf.int32)
        y_max = tf.cast(y_max * height, tf.int32)
        x_max = tf.cast(x_max * width, tf.int32)
        
        # Get the region to crop based on bbox
        crop = image[y_min:y_max, x_min:x_max]
        
        # Resize the crop to the target size
        resized_crop = tf.image.resize(crop, (224, 224))
    
        return resized_crop


    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], 224, 224, input_shape[0][-1])


# Define the model
def build_model(num_instruments, num_verbs, num_targets, num_triplets):
    image_input = Input(shape=(None, None, 3), name="image_input")
    resize_layer = Resizing(224, 224)(image_input)
    bbox_input = Input(shape=(4,), name="bbox_input")

    scale_bbox_layer = ScaleBBox(target_size=(224, 224))
    bbox_input_scaled = scale_bbox_layer(bbox_input)

    crop_to_bbox_layer = CropToBBox()
    cropped_image = crop_to_bbox_layer([image_input, bbox_input_scaled])

    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    x = base_model(cropped_image)
    x = GlobalAveragePooling2D()(x)

    x = Concatenate()([x, bbox_input_scaled])

    instrument_out = Dense(num_instruments, activation="softmax", name="instrument_id")(x)
    verb_out = Dense(num_verbs, activation="softmax", name="verb_id")(x)
    target_out = Dense(num_targets, activation="softmax", name="target_id")(x)
    triplet_out = Dense(num_triplets, activation="softmax", name="triplet_id")(x)

    return tf.keras.Model(inputs=[image_input, bbox_input], outputs=[instrument_out, verb_out, target_out, triplet_out])

# Compile the model
model = build_model(len(instrument_classes), len(verb_classes), len(target_classes), len(triplet_classes))

# Add gradient clipping
optimizer = Adam(learning_rate=1e-4, clipvalue=1.0)

model.compile(
    optimizer=optimizer,
    loss={
        "instrument_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        "verb_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        "target_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        "triplet_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    },
    metrics={
        "instrument_id": "accuracy",
        "verb_id": "accuracy",
        "target_id": "accuracy",
        "triplet_id": "accuracy",
    },
)

# Update the dataset creation with the modified class counts
train_dataset = create_tf_dataset(train_df, batch_size=16, num_instruments=len(instrument_classes), num_verbs=len(verb_classes), num_targets=len(target_classes), num_triplets=len(triplet_classes))
val_dataset = create_tf_dataset(test_df, batch_size=16, num_instruments=len(instrument_classes), num_verbs=len(verb_classes), num_targets=len(target_classes), num_triplets=len(triplet_classes))

# Force the entire model to use CPU
# with tf.device('/CPU:0'):
model.fit(train_dataset, validation_data=val_dataset, epochs=10, steps_per_epoch=10, validation_steps=50)


Epoch 1/10
Image shape before crop: Tensor("functional_13_1/crop_to_b_box_13_1/Shape:0", shape=(4,), dtype=int32)
Bounding box shape: Tensor("functional_13_1/crop_to_b_box_13_1/Shape_1:0", shape=(2,), dtype=int32)
Cropped image shape: Tensor("functional_13_1/crop_to_b_box_13_1/Shape_5:0", shape=(4,), dtype=int32)
Image shape before crop: Tensor("functional_13_1/crop_to_b_box_13_1/Shape:0", shape=(4,), dtype=int32)
Bounding box shape: Tensor("functional_13_1/crop_to_b_box_13_1/Shape_1:0", shape=(2,), dtype=int32)
Cropped image shape: Tensor("functional_13_1/crop_to_b_box_13_1/Shape_5:0", shape=(4,), dtype=int32)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - instrument_id_accuracy: 0.3123 - loss: 10.7734 - target_id_accuracy: 0.1915 - triplet_id_accuracy: 0.1147 - verb_id_accuracy: 0.1389  Image shape before crop: Tensor("functional_13_1/crop_to_b_box_13_1/Shape:0", shape=(4,), dtype=int32)
Bounding box shape: Tensor("functional_13_1/crop_to_b_box_13_1/Shape_1:0", shape=(2,), dtype=int32)
C

In [41]:
# import tensorflow as tf
# import numpy as np
# import os
# from tensorflow.keras import Model
# from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Concatenate
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import Callback
# from PIL import Image

# # Data loading and preprocessing with bbox handling


# # Define model with multi-task learning
# def build_model(num_instruments, num_verbs, num_targets, num_triplets):
#     base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
#     x = GlobalAveragePooling2D()(base_model.output)

#     # Concatenate bounding box coordinates
#     bbox_input = Input(shape=(4,), name="bbox_input")  # Bounding box coordinates
#     x = Concatenate()([x, bbox_input])  # Concatenate bbox coordinates with image features

#     # Classification branches
#     instrument_out = Dense(num_instruments, activation="softmax", name="instrument_id")(x)
#     verb_out = Dense(num_verbs, activation="softmax", name="verb_id")(x)
#     target_out = Dense(num_targets, activation="softmax", name="target_id")(x)
#     triplet_out = Dense(num_triplets, activation="softmax", name="triplet_id")(x)

#     return Model(inputs=[base_model.input, bbox_input], outputs=[instrument_out, verb_out, target_out, triplet_out])

# # Compile the model
# model = build_model(len(instrument_classes), len(verb_classes), len(target_classes), len(triplet_classes))

# # Add gradient clipping to prevent NaNs
# optimizer = Adam(learning_rate=1e-4)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipvalue=1.0)  # Clip gradients to avoid exploding gradients

# model.compile(
#     optimizer=optimizer,
#     loss={
#         "instrument_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#         "verb_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#         "target_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#         "triplet_id": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#     },
#     metrics={
#         "instrument_id": "accuracy",
#         "verb_id": "accuracy",
#         "target_id": "accuracy",
#         "triplet_id": "accuracy",
#     },
# )
# # Modify the training loop or validation to print labels and model outputs
# class NaNCheckCallback(Callback):
#     def on_batch_end(self, batch, logs=None):
#         if np.any(np.isnan(logs['loss'])):
#             print("NaN loss detected at batch:", batch)
#             self.model.stop_training = True
#         if batch == 0:  # Only check once at the beginning of training
#             print("Batch 0 outputs and labels:")
#             outputs = self.model.predict(self.model.inputs)
#             print("Outputs:", outputs)
#             print("Labels:", self.model.targets)

# # model.fit(train_dataset, validation_data=val_dataset, epochs=10, steps_per_epoch=7, callbacks=[NaNCheckCallback()])

# # Create datasets
# train_dataset = create_tf_dataset(train_df, batch_size=16, num_instruments=len(instrument_classes), 
#                                    num_verbs=len(verb_classes), num_targets=len(target_classes), 
#                                    num_triplets=len(triplet_classes))

# val_dataset = create_tf_dataset(test_df, batch_size=16, num_instruments=len(instrument_classes), 
#                                  num_verbs=len(verb_classes), num_targets=len(target_classes), 
#                                  num_triplets=len(triplet_classes))
# # print(train_dataset.type)
# # Add repeat function to avoid running out of data
# train_dataset = train_dataset.repeat()

# # Train the model with NaNCheckCallback
# model.fit(train_dataset, validation_data=val_dataset, epochs=10, steps_per_epoch=7, callbacks=[NaNCheckCallback()])


In [50]:
import tensorflow as tf
import numpy as np
import json
from PIL import Image
import os

# Assuming the trained model is loaded
# model = tf.keras.models.load_model('path_to_trained_model')

def make_predictions(test_df):
    video_predictions = {}

    for video_id in test_df['video_id'].unique():  # Loop through each video in the test data
        video_frames = test_df[test_df['video_id'] == video_id]
        frame_predictions = {}

        for _, frame_data in video_frames.iterrows():  # Iterate over each frame of the video
            frame_id = frame_data['frame_id']
            frame_path = frame_data['frame_path']
            bbox = frame_data['bbox']  # Assuming bounding boxes are in 'bbox' column
            
            # Load image from frame_path
            image = load_image(frame_path)
            
            # Preprocess the image
            processed_image = preprocess_image(image)
            
            # Make predictions for recognition (probabilities of the triplets)
            recognition_probabilities = model.predict(processed_image)  # Output is a probability vector
            
            # Get detection predictions (triplets, instrument ids, probabilities, and bounding boxes)
            detection_predictions = detect_instruments(processed_image, bbox)  # Assuming a detection function is defined

            # Format frame predictions
            frame_predictions[frame_id] = {
                "recognition": recognition_probabilities.tolist(),
                "detection": detection_predictions
            }

        # Store predictions for the video
        video_predictions[video_id] = frame_predictions
    
    return video_predictions

def load_image(frame_path):
    """Load image from the given path."""
    try:
        image = Image.open(frame_path)
        image = image.convert("RGB")  # Ensure image is in RGB format
        return np.array(image)
    except Exception as e:
        print(f"Error loading image {frame_path}: {e}")
        return None

def preprocess_image(image):
    """Preprocess the image (resize, normalize, etc.)."""
    if image is not None:
        # Resize image to the required input shape
        image = tf.image.resize(image, (224, 224))  
        image = image / 255.0  # Normalize if needed
        return np.expand_dims(image, axis=0)  # Add batch dimension
    return np.zeros((1, 224, 224, 3))  # Return a dummy image if loading failed

def detect_instruments(image, bbox):
    """Detect instruments based on bounding box and image."""
    detections = []
    if bbox:  # If bounding box exists, use it for detection
        for triplet in range(100):  # Assuming you have 100 triplets
            tool_id = triplet  # Replace with actual logic for tool_id prediction
            tool_prob = np.random.rand()  # Random probability for now, replace with actual prediction
            detection = {
                "triplet": triplet,
                "instrument": [tool_id, tool_prob, *bbox]
            }
            detections.append(detection)
    return detections

# Example: Generate the prediction results
test_predictions = make_predictions(test_df)

# Save the results in the required JSON format
output_filename = "MODEL_NAME.json"  # Replace with your model name
with open(output_filename, 'w') as f:
    json.dump(test_predictions, f, indent=4)

print(f"Predictions saved to {output_filename}")


KeyError: 'frame_id'

In [103]:
!nvidia-smi


Mon Jan 13 19:41:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             32W /  250W |    3151MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [102]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [100]:
print(train_df.columns)




Index(['video_id', 'frame_path', 'triplet_id', 'instrument_id', 'verb_id',
       'target_id', 'phase_id', 'bbox'],
      dtype='object')


In [45]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class SurgicalDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, test_dataset, batch_size=32):
        super().__init__()
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        # Create DataLoader for training dataset
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        # Create DataLoader for validation/test dataset
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)

# # Example of how you can use it
# # Make sure you define `train_dataset` and `test_dataset` before using the data module
# train_dataset = ...  # Define your training dataset
# test_dataset = ...   # Define your testing dataset

data_module = SurgicalDataModule(train_dataset, test_dataset, batch_size=16)


In [46]:
import os
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

# Disable CUDA completely
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Monitor validation loss
    dirpath="./checkpoints",  # Save to this directory
    filename="triplet_model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,  # Save only the best model
    mode="min",  # Minimize validation loss
)

# Define the Lightning model
class MultiTaskTripletLightningModel(pl.LightningModule):
    def __init__(self, num_instruments, num_verbs, num_targets, num_triplets):
        super().__init__()
        # Ensure the model is initialized for CPU
        self.model = ...  # Define your model layers

    def forward(self, x):
        x = x.to("cpu")  # Ensure input tensor is on the CPU
        return self.model(x)

# Instantiate the Lightning model
lightning_model = MultiTaskTripletLightningModel(
    num_instruments=len(instrument_classes),
    num_verbs=len(verb_classes),
    num_targets=len(target_classes),
    num_triplets=len(triplet_classes)
)

# Move model to CPU explicitly
lightning_model.to("cpu")

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="cpu",  # Ensure CPU usage
    devices=1,           # Use only one CPU
    log_every_n_steps=20,
    enable_checkpointing=False,  # Disable checkpointing
    enable_progress_bar=True,  # Enable progress bar
)

# Fit the model with the data module
trainer.fit(lightning_model, datamodule=data_module)


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.